In [1]:
import pandas as pd
import numpy as np 

from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error as mse

In [2]:
df = pd.read_csv('../../datasets/synthesized_loan_data.csv')

In [3]:
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,1,1.015491,0,0,60,2263.426739,120.264574,362.034926,0.999618,2,1
1,1,1,2.793550,1,2,299,207.078743,97.333547,185.855127,0.994450,1,1
2,0,0,1.020575,0,1,497,-222.105153,98.957504,361.411489,0.999132,0,1
3,1,1,0.967628,0,0,35,1874.200797,141.121981,360.427013,0.999273,2,1
4,1,1,1.929354,0,0,340,47.080906,322.936778,360.013915,0.979882,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1,1,2.118012,0,0,248,2693.049737,176.172212,360.665439,1.007047,1,1
3996,0,0,-0.002574,0,0,249,1949.728061,117.277610,361.044262,0.986177,1,1
3997,1,1,0.010152,0,0,300,2882.202092,136.098872,361.722296,0.994603,0,1
3998,0,1,-0.002625,0,0,385,182.631094,191.733185,360.723093,0.989029,1,0


In [4]:
df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [5]:
train_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area']

x = df[train_cols]
y = df['Loan_Status']

In [6]:
x

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,1,1,1.015491,0,0,60,2263.426739,120.264574,362.034926,0.999618,2
1,1,1,2.793550,1,2,299,207.078743,97.333547,185.855127,0.994450,1
2,0,0,1.020575,0,1,497,-222.105153,98.957504,361.411489,0.999132,0
3,1,1,0.967628,0,0,35,1874.200797,141.121981,360.427013,0.999273,2
4,1,1,1.929354,0,0,340,47.080906,322.936778,360.013915,0.979882,0
...,...,...,...,...,...,...,...,...,...,...,...
3995,1,1,2.118012,0,0,248,2693.049737,176.172212,360.665439,1.007047,1
3996,0,0,-0.002574,0,0,249,1949.728061,117.277610,361.044262,0.986177,1
3997,1,1,0.010152,0,0,300,2882.202092,136.098872,361.722296,0.994603,0
3998,0,1,-0.002625,0,0,385,182.631094,191.733185,360.723093,0.989029,1


In [7]:
y

0       1
1       1
2       1
3       1
4       1
       ..
3995    1
3996    1
3997    1
3998    0
3999    1
Name: Loan_Status, Length: 4000, dtype: int64

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=20)

In [66]:
cb_sy_grid = CatBoostClassifier(n_estimators=100,
                       loss_function='Logloss',
                       learning_rate=0.1,
                       #objective='CrossEntropy',
                       #colsample_bylevel= 0.06,
                        task_type='CPU',
                       random_state=1,
                       verbose=False)

In [74]:
grid_loan = {
    #'max_depth' : [3, 4, 5],
    'n_estimators' : [100, 200, 300],
    #'learning_rate' : [0.01, 0.05, 0.1],
    'subsample' : [0.1, 0.2, 0.4, 0.8],
    'colsample_bylevel' : [0.01, 0.05, 0.09]
}

In [75]:
grid_cat = GridSearchCV(
    estimator = cb_sy_grid,
    param_grid= grid_loan,
    scoring= 'accuracy',
    cv = 5
)

In [76]:
grid_cat.fit(x_train, y_train)

C:\Users\Wessware\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
180 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Wessware\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Wessware\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\loca

TypeError: __init__() got an unexpected keyword argument 'sub_sample'

In [ ]:
loan_predicted = grid_cat.predict(x_test)

In [ ]:
model_rmse_grid = np.sqrt(mse(y_test, loan_predicted))

In [ ]:
model_rmse_grid


0.27688746209726917

In [ ]:
#large grid 0.279888
#small grid 0.292973
#smaller grid_0 0.284312
#smaller grid_1 0.2828427
#smaller grid_2 0.276887